In [1]:
import module_nn
import util
import torch
import os
import soundfile as sf
from pathlib import Path
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import pywt
from scipy.io import wavfile
import librosa
from skimage.restoration import denoise_wavelet
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from denoise import AudioDeNoise 

In [2]:
LIST_ALL_AUDIOS_GENERATE_DENOISED = util.load_variables('variables/list_for_denoised_audios_speakers.pkl')
LIST_ALL_AUDIOS_GENERATE_DENOISED[1]

['audio_with_noise/-9/nonstationary_crowd_party_adult_med/196-122150-0019.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1723-141149-0016.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1723-141149-0011.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1898-145702-0012.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/196-122150-0034.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1898-145720-0003.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1723-141149-0074.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1963-147036-0005.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1898-145715-0002.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1898-145702-0017.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1867-154071-0006.flac',
 'audio_with_noise/-9/nonstationary_crowd_party_adult_med/1898-145720-0009.flac',
 'audio_with_noise

In [3]:
len(LIST_ALL_AUDIOS_GENERATE_DENOISED)

98

In [4]:
LIST_ALL_AUDIOS_GENERATE_CLEAN = util.load_variables('variables/list_clean_audios_speakers.pkl')
LIST_ALL_AUDIOS_GENERATE_CLEAN[1]

['data/Speakers/train-clean-100/196/196-122150-0019.flac',
 'data/Speakers/train-clean-100/1723/1723-141149-0016.flac',
 'data/Speakers/train-clean-100/1723/1723-141149-0011.flac',
 'data/Speakers/train-clean-100/1898/1898-145702-0012.flac',
 'data/Speakers/train-clean-100/196/196-122150-0034.flac',
 'data/Speakers/train-clean-100/1898/1898-145720-0003.flac',
 'data/Speakers/train-clean-100/1723/1723-141149-0074.flac',
 'data/Speakers/train-clean-100/1963/1963-147036-0005.flac',
 'data/Speakers/train-clean-100/1898/1898-145715-0002.flac',
 'data/Speakers/train-clean-100/1898/1898-145702-0017.flac',
 'data/Speakers/train-clean-100/1867/1867-154071-0006.flac',
 'data/Speakers/train-clean-100/1898/1898-145720-0009.flac',
 'data/Speakers/train-clean-100/1841/1841-159771-0012.flac',
 'data/Speakers/train-clean-100/196/196-122152-0015.flac',
 'data/Speakers/train-clean-100/1723/1723-141149-0056.flac',
 'data/Speakers/train-clean-100/1743/1743-142912-0033.flac',
 'data/Speakers/train-clean-10

In [5]:
len(LIST_ALL_AUDIOS_GENERATE_CLEAN)

98

In [6]:
CLEAN_AUDIO = []
for i in range(len(LIST_ALL_AUDIOS_GENERATE_CLEAN)):
    CLEAN_AUDIO.extend(LIST_ALL_AUDIOS_GENERATE_CLEAN[i])

In [7]:
#Load model for evaluation
model = module_nn.Generator()
model.load_state_dict(torch.load('models_regression/G_model_NOISE_DA_WITH_OT_LOSS_SourceTarget_ALL_NOISES_dB_Generator_FINAL.pt'))
model.eval()

Generator(
  (blstm): LSTM(257, 1024, num_layers=2, batch_first=True, bidirectional=True)
  (LReLU): LeakyReLU(negative_slope=0.3)
  (ReLU): ReLU()
  (Dropout): Dropout(p=0.0, inplace=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=257, bias=True)
)

In [8]:
PATH_AUDIOS_DENOISED_DNN = 'audios_denoised_dnn/'
PATH_AUDIOS_DENOISED_DNN_WT = 'audios_denoised_wt/'

In [9]:
SCORES_BEFORE_PESQ = []
SCORES_BEFORE_STOI = []

SCORES_DNN_PESQ = []
SCORES_DNN_STOI = []

SCORES_DNN_TW_PESQ = []
SCORES_DNN_TW_STOI = []


PATH_CLEAN_AUDIO = []
PATH_DENOISED_AUDIO_DNN = []
PATH_DENOISED_AUDIO_DNN_TW = []

iterator = 0
for snr_noise_speaker in LIST_ALL_AUDIOS_GENERATE_DENOISED:
    BEFORE_PESQ = []
    BEFORE_STOI = []
    
    PESQ_DNN = []
    STOI_DNN = []
    
    PESQ_DNN_TW = []
    STOI_DNN_TW = []

    DNN_TW_PATH_DENOISED_AUDIO = []
    DNN_PATH_DENOISED_AUDIO = []
    PATH_AUDIO_CLEAN = []
    for file in snr_noise_speaker:
        print(file.split('/')[1], file.split('/')[2], file.split('/')[3])
        Path(os.path.join(PATH_AUDIOS_DENOISED_DNN,"/".join(file.split('/')[1:len(file.split('/'))-1]))).mkdir(parents=True, exist_ok=True)
        Path(os.path.join(PATH_AUDIOS_DENOISED_DNN_WT,"/".join(file.split('/')[1:len(file.split('/'))-1]))).mkdir(parents=True, exist_ok=True)
        noise_wave, sr = librosa.load(file, sr=16000)
        
        X_spec, noisy_phase, noisy_len = util.make_spectrum(y=noise_wave)
        X_spec = torch.from_numpy(X_spec.transpose()).to('cpu').unsqueeze(0)
        y_spec_pred = model(X_spec).detach().cpu().numpy().squeeze(0)# / self.args.amplify
        y_pred = util.recons_spec_phase(y_spec_pred.transpose(), noisy_phase, noisy_len)
        sf.write(os.path.join(PATH_AUDIOS_DENOISED_DNN,"/".join(file.split('/')[1:len(file.split('/'))])), y_pred, samplerate=16000, subtype='PCM_16')
        DNN_PATH_DENOISED_AUDIO.append(os.path.join(PATH_AUDIOS_DENOISED_DNN,"/".join(file.split('/')[1:len(file.split('/'))])))
        clean, _ = util.read_audio(CLEAN_AUDIO[iterator], 16000)
        PATH_AUDIO_CLEAN.append(CLEAN_AUDIO[iterator])
        
        pesq_before, stoi_before, _, _, _ = util.eval_audio(clean, noise_wave)
        print("(%d) - B - PESQ: %.3f \t STOI: %.3f" % (iterator+1, pesq_before, stoi_before))
        
        pesq_dnn, stoi_dnn, _, _, _ = util.eval_audio(clean, y_pred)
        print("(%d) - D - PESQ: %.3f \t STOI: %.3f" % (iterator+1, pesq_dnn, stoi_dnn))
        
        sigma_est = estimate_sigma(y_pred, average_sigmas=True)
        
        #im_visushrink4 = denoise_wavelet(y_pred, wavelet='haar', wavelet_levels=9,
        #                         method='VisuShrink', mode='soft',
        #                         sigma=sigma_est/4, rescale_sigma=True)
        #
        
        #im_visushrink4 = denoise_wavelet(y_pred, wavelet='db1', wavelet_levels=3,
        #                         method='VisuShrink', mode='soft',
        #                         sigma=sigma_est/4, rescale_sigma=True)
        
        im_visushrink4 = denoise_wavelet(y_pred, wavelet='db1', wavelet_levels=9,
                                 method='VisuShrink', mode='soft',
                                 sigma=sigma_est/4, rescale_sigma=True)
        
        #im_visushrink4 = denoise_wavelet(y_pred, wavelet='rbio3.3', wavelet_levels=3,
        #                         method='VisuShrink', mode='hard',
        #                         sigma=sigma_est/4, rescale_sigma=True)
        
        sf.write(os.path.join(PATH_AUDIOS_DENOISED_DNN_WT,"/".join(file.split('/')[1:len(file.split('/'))])), im_visushrink4, samplerate=16000, subtype='PCM_16')
        DNN_TW_PATH_DENOISED_AUDIO.append(os.path.join(PATH_AUDIOS_DENOISED_DNN_WT,"/".join(file.split('/')[1:len(file.split('/'))])))
        
        pesq_dnn_wt, stoi_dnn_wt, _, _, _ = util.eval_audio(clean, im_visushrink4)
        print("(%d) - T - PESQ: %.3f \t STOI: %.3f" % (iterator+1, pesq_dnn_wt, stoi_dnn_wt))
        
        BEFORE_PESQ.append(pesq_before)
        BEFORE_STOI.append(stoi_before)
        
        PESQ_DNN.append(pesq_dnn)
        STOI_DNN.append(stoi_dnn)
        
        PESQ_DNN_TW.append(pesq_dnn_wt)
        STOI_DNN_TW.append(stoi_dnn_wt)
        
        iterator+=1
    
    print('===================================================================================')
    print('BEFORE_PESQ: '+ str(np.sum(np.array(BEFORE_PESQ))/(len(BEFORE_PESQ))) + '\t PESQ_DNN: ' + str(np.sum(np.array(PESQ_DNN))/(len(np.array(PESQ_DNN)))) + '\t PESQ_DNN_WT: ' + str(np.sum(np.array(PESQ_DNN_TW))/(len(np.array(PESQ_DNN_TW)))))
    print('BEFORE_STOI: '+ str(np.sum(np.array(BEFORE_STOI))/(len(BEFORE_STOI))) + '\t STOI_DNN: ' + str(np.sum(np.array(STOI_DNN))/(len(np.array(STOI_DNN)))) + '\t STOI_DNN_WT: ' + str(np.sum(np.array(STOI_DNN_TW))/(len(np.array(STOI_DNN_TW)))))
    print('===================================================================================')
    
    SCORES_BEFORE_PESQ.append(BEFORE_PESQ)
    SCORES_BEFORE_STOI.append(BEFORE_STOI)
    #SCORES_BEFORE_SNR.append(BEFORE_SNR)
    #SCORES_BEFORE_SI_SDR.append(BEFORE_SI_SDR)
    #SCORES_BEFORE_SI_SNR.append(BEFORE_SI_SNR)
    
    SCORES_DNN_PESQ.append(PESQ_DNN)
    SCORES_DNN_STOI.append(STOI_DNN)
    #SCORES_DNN_SNR.append(SNR_DNN)
    #SCORES_DNN_SI_SDR.append(SI_SDR_DNN)
    #SCORES_DNN_SI_SNR.append(SI_SNR_DNN)
    
    SCORES_DNN_TW_PESQ.append(PESQ_DNN_TW)
    SCORES_DNN_TW_STOI.append(STOI_DNN_TW)

    PATH_DENOISED_AUDIO_DNN_TW.append(DNN_TW_PATH_DENOISED_AUDIO)
    PATH_DENOISED_AUDIO_DNN.append(DNN_PATH_DENOISED_AUDIO)
    PATH_CLEAN_AUDIO.append(PATH_AUDIO_CLEAN)
    #breakDB

-9 nonstationary_babycry 196-122150-0019.flac
(1) - B - PESQ: 1.084 	 STOI: 0.672
(1) - D - PESQ: 1.892 	 STOI: 0.889
(1) - T - PESQ: 2.088 	 STOI: 0.889
-9 nonstationary_babycry 1723-141149-0016.flac
(2) - B - PESQ: 1.232 	 STOI: 0.669
(2) - D - PESQ: 1.883 	 STOI: 0.861
(2) - T - PESQ: 2.030 	 STOI: 0.855
-9 nonstationary_babycry 1723-141149-0011.flac
(3) - B - PESQ: 1.184 	 STOI: 0.665
(3) - D - PESQ: 1.791 	 STOI: 0.849
(3) - T - PESQ: 1.996 	 STOI: 0.846
-9 nonstationary_babycry 1898-145702-0012.flac
(4) - B - PESQ: 1.305 	 STOI: 0.579
(4) - D - PESQ: 1.414 	 STOI: 0.810
(4) - T - PESQ: 1.625 	 STOI: 0.812
-9 nonstationary_babycry 196-122150-0034.flac
(5) - B - PESQ: 1.942 	 STOI: 0.687
(5) - D - PESQ: 1.906 	 STOI: 0.891
(5) - T - PESQ: 2.081 	 STOI: 0.891
-9 nonstationary_babycry 1898-145720-0003.flac
(6) - B - PESQ: 1.280 	 STOI: 0.615
(6) - D - PESQ: 1.521 	 STOI: 0.766
(6) - T - PESQ: 1.690 	 STOI: 0.767
-9 nonstationary_babycry 1723-141149-0074.flac
(7) - B - PESQ: 1.225 	 S

/tmp/ipykernel_4569/677223387.py:33: UserWarning: PySoundFile failed. Trying audioread instead.
  noise_wave, sr = librosa.load(file, sr=16000)
/home/iemmanuelvm/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/iemmanuelvm/Documents/Denoised Audio 1/util.py:98: RuntimeWarning: invalid value encountered in divide
  enhanced = denoise_audio / abs(denoise_audio).max()


(1910) - B - PESQ: -0.500 	 STOI: 0.000
(1910) - D - PESQ: 1.046 	 STOI: 0.399
(1910) - T - PESQ: 1.046 	 STOI: 0.399
-6 nonstationary_people 1867-154071-0006.flac
(1911) - B - PESQ: 2.418 	 STOI: 0.904
(1911) - D - PESQ: 3.049 	 STOI: 0.968
(1911) - T - PESQ: 3.004 	 STOI: 0.965
-6 nonstationary_people 1898-145720-0009.flac
(1912) - B - PESQ: 2.587 	 STOI: 0.789
(1912) - D - PESQ: 2.652 	 STOI: 0.873
(1912) - T - PESQ: 2.873 	 STOI: 0.878
-6 nonstationary_people 1841-159771-0012.flac
(1913) - B - PESQ: 1.764 	 STOI: 0.817
(1913) - D - PESQ: 2.427 	 STOI: 0.927
(1913) - T - PESQ: 2.410 	 STOI: 0.919
-6 nonstationary_people 196-122152-0015.flac
(1914) - B - PESQ: 1.809 	 STOI: 0.793
(1914) - D - PESQ: 2.462 	 STOI: 0.929
(1914) - T - PESQ: 2.642 	 STOI: 0.928
-6 nonstationary_people 1723-141149-0056.flac
(1915) - B - PESQ: 1.893 	 STOI: 0.771
(1915) - D - PESQ: 2.404 	 STOI: 0.884
(1915) - T - PESQ: 2.545 	 STOI: 0.864
-6 nonstationary_people 1743-142912-0033.flac
(1916) - B - PESQ: 1.8

/tmp/ipykernel_4569/677223387.py:33: UserWarning: PySoundFile failed. Trying audioread instead.
  noise_wave, sr = librosa.load(file, sr=16000)
/home/iemmanuelvm/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/iemmanuelvm/Documents/Denoised Audio 1/util.py:98: RuntimeWarning: invalid value encountered in divide
  enhanced = denoise_audio / abs(denoise_audio).max()


(1943) - B - PESQ: -0.500 	 STOI: 0.000
(1943) - D - PESQ: 1.025 	 STOI: 0.352
(1943) - T - PESQ: 1.026 	 STOI: 0.352
-6 nonstationary_people 19-227-0038.flac
(1944) - B - PESQ: 1.726 	 STOI: 0.767
(1944) - D - PESQ: 2.585 	 STOI: 0.860
(1944) - T - PESQ: 2.521 	 STOI: 0.814
-6 nonstationary_people 1898-145724-0005.flac
(1945) - B - PESQ: 1.241 	 STOI: 0.797
(1945) - D - PESQ: 1.932 	 STOI: 0.864
(1945) - T - PESQ: 1.789 	 STOI: 0.843
-6 nonstationary_people 19-227-0018.flac
(1946) - B - PESQ: 1.829 	 STOI: 0.742
(1946) - D - PESQ: 2.537 	 STOI: 0.857
(1946) - T - PESQ: 2.468 	 STOI: 0.814
-6 nonstationary_people 1867-154071-0031.flac
(1947) - B - PESQ: 2.017 	 STOI: 0.842
(1947) - D - PESQ: 2.614 	 STOI: 0.933
(1947) - T - PESQ: 2.654 	 STOI: 0.931
-6 nonstationary_people 1926-143879-0034.flac
(1948) - B - PESQ: 1.706 	 STOI: 0.821
(1948) - D - PESQ: 2.274 	 STOI: 0.931
(1948) - T - PESQ: 2.223 	 STOI: 0.926
-6 nonstationary_people 1723-141149-0052.flac
(1949) - B - PESQ: 2.194 	 STOI

...

(6138) - B - PESQ: -0.500 	 STOI: 0.000
(6138) - D - PESQ: 1.039 	 STOI: 0.433
(6138) - T - PESQ: 1.040 	 STOI: 0.433
3 nonstationary_people 1926-147979-0030.flac
(6139) - B - PESQ: 2.329 	 STOI: 0.908
(6139) - D - PESQ: 3.005 	 STOI: 0.953
(6139) - T - PESQ: 2.949 	 STOI: 0.948
3 nonstationary_people 1926-147979-0013.flac
(6140) - B - PESQ: 2.325 	 STOI: 0.887
(6140) - D - PESQ: 2.822 	 STOI: 0.928
(6140) - T - PESQ: 2.854 	 STOI: 0.913
3 nonstationary_people 1723-141149-0098.flac
(6141) - B - PESQ: 2.607 	 STOI: 0.901
(6141) - D - PESQ: 2.554 	 STOI: 0.930
(6141) - T - PESQ: 2.829 	 STOI: 0.920
3 nonstationary_people 1926-143879-0037.flac
(6142) - B - PESQ: 2.002 	 STOI: 0.880
(6142) - D - PESQ: 2.804 	 STOI: 0.952
(6142) - T - PESQ: 2.880 	 STOI: 0.951
3 nonstationary_people 1963-142393-0028.flac


/tmp/ipykernel_4569/677223387.py:33: UserWarning: PySoundFile failed. Trying audioread instead.
  noise_wave, sr = librosa.load(file, sr=16000)
/home/iemmanuelvm/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/iemmanuelvm/Documents/Denoised Audio 1/util.py:98: RuntimeWarning: invalid value encountered in divide
  enhanced = denoise_audio / abs(denoise_audio).max()


(6143) - B - PESQ: -0.500 	 STOI: 0.000
(6143) - D - PESQ: 1.025 	 STOI: 0.352
(6143) - T - PESQ: 1.026 	 STOI: 0.352
3 nonstationary_people 19-227-0038.flac
(6144) - B - PESQ: 2.859 	 STOI: 0.796
(6144) - D - PESQ: 3.094 	 STOI: 0.861
(6144) - T - PESQ: 3.142 	 STOI: 0.860
3 nonstationary_people 1898-145724-0005.flac
(6145) - B - PESQ: 2.152 	 STOI: 0.864
(6145) - D - PESQ: 2.241 	 STOI: 0.899
(6145) - T - PESQ: 2.138 	 STOI: 0.881
3 nonstationary_people 19-227-0018.flac
(6146) - B - PESQ: 3.040 	 STOI: 0.768
(6146) - D - PESQ: 3.261 	 STOI: 0.844
(6146) - T - PESQ: 3.332 	 STOI: 0.842
3 nonstationary_people 1867-154071-0031.flac
(6147) - B - PESQ: 2.661 	 STOI: 0.921
(6147) - D - PESQ: 3.018 	 STOI: 0.963
(6147) - T - PESQ: 3.064 	 STOI: 0.962
3 nonstationary_people 1926-143879-0034.flac
(6148) - B - PESQ: 2.367 	 STOI: 0.887
(6148) - D - PESQ: 2.850 	 STOI: 0.953
(6148) - T - PESQ: 2.789 	 STOI: 0.948
3 nonstationary_people 1723-141149-0052.flac
(6149) - B - PESQ: 2.384 	 STOI: 0.85

/tmp/ipykernel_4569/677223387.py:33: UserWarning: PySoundFile failed. Trying audioread instead.
  noise_wave, sr = librosa.load(file, sr=16000)
/home/iemmanuelvm/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/iemmanuelvm/Documents/Denoised Audio 1/util.py:98: RuntimeWarning: invalid value encountered in divide
  enhanced = denoise_audio / abs(denoise_audio).max()


(8943) - B - PESQ: -0.500 	 STOI: 0.000
(8943) - D - PESQ: 1.025 	 STOI: 0.352
(8943) - T - PESQ: 1.026 	 STOI: 0.352
9 nonstationary_people 19-227-0038.flac
(8944) - B - PESQ: 2.902 	 STOI: 0.868
(8944) - D - PESQ: 3.246 	 STOI: 0.893
(8944) - T - PESQ: 3.257 	 STOI: 0.865
9 nonstationary_people 1898-145724-0005.flac
(8945) - B - PESQ: 2.318 	 STOI: 0.902
(8945) - D - PESQ: 2.046 	 STOI: 0.894
(8945) - T - PESQ: 1.869 	 STOI: 0.873
9 nonstationary_people 19-227-0018.flac
(8946) - B - PESQ: 2.948 	 STOI: 0.887
(8946) - D - PESQ: 3.342 	 STOI: 0.918
(8946) - T - PESQ: 3.335 	 STOI: 0.878
9 nonstationary_people 1867-154071-0031.flac
(8947) - B - PESQ: 2.909 	 STOI: 0.966
(8947) - D - PESQ: 3.217 	 STOI: 0.975
(8947) - T - PESQ: 3.296 	 STOI: 0.974
9 nonstationary_people 1926-143879-0034.flac
(8948) - B - PESQ: 2.889 	 STOI: 0.932
(8948) - D - PESQ: 3.200 	 STOI: 0.971
(8948) - T - PESQ: 3.078 	 STOI: 0.966
9 nonstationary_people 1723-141149-0052.flac
(8949) - B - PESQ: 2.961 	 STOI: 0.91

In [10]:
util.generate_pkl('variables/', SCORES_BEFORE_PESQ, 'SCORES_BEFORE_PESQ')
util.generate_pkl('variables/', SCORES_BEFORE_STOI, 'SCORES_BEFORE_STOI')

util.generate_pkl('variables/', SCORES_DNN_PESQ, 'SCORES_DNN_PESQ')
util.generate_pkl('variables/', SCORES_DNN_STOI, 'SCORES_DNN_STOI')

util.generate_pkl('variables/', SCORES_DNN_TW_PESQ, 'SCORES_DNN_TW_PESQ')
util.generate_pkl('variables/', SCORES_DNN_TW_STOI, 'SCORES_DNN_TW_STOI')

util.generate_pkl('variables/', PATH_DENOISED_AUDIO_DNN_TW, 'PATH_DENOISED_AUDIO_DNN_TW')
util.generate_pkl('variables/', PATH_DENOISED_AUDIO_DNN, 'PATH_DENOISED_AUDIO_DNN')
util.generate_pkl('variables/', PATH_CLEAN_AUDIO, 'PATH_CLEAN_AUDIO')

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
def get_metrics_for_each_snr(LIST_PESQ, LIST_STOI):
    PESQ_STORE_AVG = []
    STOI_STORE_AVG = []
    for idx, (PESQ, STOI) in enumerate(zip(LIST_PESQ, LIST_STOI)):
        PESQ_AVG = np.sum(np.array(PESQ))/len(np.array(PESQ))
        STOI_AVG = np.sum(np.array(STOI))/len(np.array(STOI))
        PESQ_STORE_AVG.append(PESQ_AVG)
        STOI_STORE_AVG.append(STOI_AVG)
    return PESQ_STORE_AVG, STOI_STORE_AVG

In [ ]:
AVG_SNR_SCORES_BEFORE_PESQ, AVG_SNR_SCORES_BEFORE_STOI = get_metrics_for_each_snr(SCORES_BEFORE_PESQ, SCORES_BEFORE_STOI)
AVG_SNR_SCORES_DNN_PESQ, AVG_SNR_SCORES_DNN_STOI = get_metrics_for_each_snr(SCORES_DNN_PESQ, SCORES_DNN_STOI)
AVG_SNR_SCORES_DNN_WT_PESQ, AVG_SNR_SCORES_DNN_WT_STOI = get_metrics_for_each_snr(SCORES_DNN_WT_PESQ, SCORES_DNN_WT_STOI)

In [ ]:
def plot_stoi_scores(df, **config):
    plt.style.use('ggplot')
    cmap = cm.get_cmap('magma')
    # plot the DataFrame
    ax = df.plot(x='Noise', xlabel='SNR (dB)', ylabel='STOI Scores',
            kind='bar',
            stacked=False,
            title='Short-Time Objective Intelligibility - ' + str(config['config']), cmap=cmap)
    # rotate the x-axis label by 45 degrees
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.legend(bbox_to_anchor=(1.05, 1), loc=0)
    # show the plot
    plt.show()

In [ ]:
#print(plt.colormaps())

In [ ]:
def plot_pesq_scores(df, **config):
    plt.style.use('ggplot')
    cmap = cm.get_cmap('gnuplot')

    # plot the DataFrame
    ax = df.plot(x='Noise', xlabel='SNR (dB)', ylabel='PESQ Scores',
            kind='bar',
            stacked=False,
            title='Perceptual Evaluation of Speech Quality - ' + str(config['config']), cmap=cmap)

    # rotate the x-axis label by 45 degrees
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.legend(bbox_to_anchor=(1.05, 1), loc=0)

    # show the plot
    plt.show()

In [ ]:
def get_scores_each_snr_noise_to_plot(data_snr_before, data_snr_dnn, data_snr_dnn_wt):
    engine = []
    cabin = []
    pink = []
    car = []
    wind1 = []
    CROWD_PARTY_ADULT_MED = []
    helicopter = []
    babycry = []
    cafeteria_babble = []
    count = 0
    for idx, (noise_, dnn_, dnn_wt_) in enumerate(zip(data_snr_before, data_snr_dnn, data_snr_dnn_wt)):
        print(idx)
        print(idx, noise_, dnn_, dnn_wt_)

        if(count == 0):
            print('save engine')
            engine.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 1):
            print('save cabin')
            cabin.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 2):
            print('save pink')
            pink.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 3):
            print('save car')
            car.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 4):
            print('save wind1')
            wind1.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 5):
            print('save CROWD_PARTY_ADULT_MED')
            CROWD_PARTY_ADULT_MED.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 6):
            print('save helicopter')
            helicopter.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 7):
            print('save babycry')
            babycry.append([idx, noise_, dnn_, dnn_wt_])
        if(count == 8):
            print('save cafeteria_babble')
            cafeteria_babble.append([idx, noise_, dnn_, dnn_wt_])

        if(count==len(noises)-1):
            print(noises[count])
            count=0
        else:
            print(noises[count])
            count += 1
    SCORES_FINAL = [engine, cabin, pink, car, wind1, CROWD_PARTY_ADULT_MED, helicopter, babycry, cafeteria_babble]
    return SCORES_FINAL

In [ ]:
FINAL_STOI_SCORES = get_scores_each_snr_noise_to_plot(AVG_SNR_SCORES_BEFORE_STOI, AVG_SNR_SCORES_DNN_STOI, AVG_SNR_SCORES_DNN_WT_STOI)

In [ ]:
FINAL_PESQ_SCORES = get_scores_each_snr_noise_to_plot(AVG_SNR_SCORES_BEFORE_PESQ, AVG_SNR_SCORES_DNN_PESQ, AVG_SNR_SCORES_DNN_WT_PESQ)

In [ ]:
NOISES_TITLE = ['Source Domain - Type of noise: Engine',
                'Source Domain - Type of noise: Cabin',
                'Source Domain - Type of noise: Pink',
                'Source Domain - Type of noise: Car',
                'Source Domain - Type of noise: Wind',
                'Target Domain - Type of noise: Crowd Party Adult Med',
                'Target Domain - Type of noise: Helicopter',
                'Target Domain - Type of noise: Baby Cry',
                'Target Domain - Type of noise: Cafeteria Babble']

In [ ]:
level_snr = ['-9','-6','-3','0','3','6','9','12']
count = 0
for i in range(len(FINAL_STOI_SCORES)):
    for j in range(len(FINAL_STOI_SCORES[i])):
        #print(FINAL_STOI_SCORES[i][j][0])
        #print(level_snr[count])
        FINAL_STOI_SCORES[i][j][0] = level_snr[count]
        count += 1
    count = 0

In [ ]:
level_snr = ['-9','-6','-3','0','3','6','9','12']
count = 0
for i in range(len(FINAL_PESQ_SCORES)):
    for j in range(len(FINAL_PESQ_SCORES[i])):
        #print(FINAL_STOI_SCORES[i][j][0])
        #print(level_snr[count])
        FINAL_PESQ_SCORES[i][j][0] = level_snr[count]
        count += 1
    count = 0

In [ ]:
columns = ['Noise', 'Audio + Noise', 'Applying DNN', 'Applying DNN+WT']
count_title = 0
for data in FINAL_STOI_SCORES:
    df = pd.DataFrame(data, columns=columns)
    print(df)
    df.to_csv('dataframes/'+str(NOISES_TITLE[count_title]), index=False)
    plot_stoi_scores(df, config=NOISES_TITLE[count_title])
    count_title += 1

In [ ]:
count_title = 0
for data in FINAL_PESQ_SCORES:
    df = pd.DataFrame(data, columns=columns)
    df.to_csv('dataframes/'+str(NOISES_TITLE[count_title]), index=False)
    print(df)
    plot_pesq_scores(df, config=NOISES_TITLE[count_title])
    count_title+=1

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
c1, _ = librosa.load('audio_with_noise/-9/nonstationary_babycry/1723-141149-0011.flac')
c2, _ = librosa.load('audios_denoised_dnn/-9/nonstationary_babycry/1723-141149-0011.flac')
c3, _ = librosa.load('data/LibriSpeech/train-clean-100/1723/141149/1723-141149-0011.flac')

In [ ]:
plt.plot(c1)
plt.plot(c3)
plt.plot(c2)
